In [1]:
from IPython import display
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
import omegaconf
from gymnasium import spaces
import gymnasium as gym


import mbrl.models as models
from mbrl.models import Model
import mbrl.planning as planning
import mbrl.util.common as common_util
import mbrl.util as util
from pyoperon.sklearn import SymbolicRegressor


%load_ext autoreload
%autoreload 2

mpl.rcParams.update({"font.size": 16})

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [140]:
def term_fn(a, next_obs):
    #threshold = 10
    #not_done = (next_obs > -threshold)* (next_obs < threshold)
    #done = ~not_done
    #done = done[:, None]
    return False

def reward_fn(a, next_obs):
    return torch.cos(2 * torch.pi * next_obs) * torch.exp(torch.abs(next_obs) / 3)

In [141]:
class Simple1DMDP(gym.Env):
    def __init__(self):
        super(Simple1DMDP, self).__init__()
        
        # define the action space
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        
        # define the observation space: continuous single dimension for position
        self.observation_space = spaces.Box(low=-10, high=10, shape=(1,), dtype=np.float32)
        # Initialize state and episode length
        self.state = 0.0
        self.episode_length = 10
        self.current_step = 0

    def reset(self, seed=None):
        # reset the state to 0 and the step counter
        self.state = 0.0
        self.current_step = 0
        return torch.FloatTensor([self.state]), {}

    def step(self, action):  
        # update state based on action
        self.state += action
        # clip in the case we go outside of [-10,10]
        actual_new_state = self.state
        self.state = np.clip(self.state, -10.,10.)
        
        # calculate reward
        #reward = torch.cos(2 * torch.pi * self.state) * torch.exp(torch.abs(self.state) / 3)
        reward = reward_fn(torch.from_numpy(action), torch.from_numpy(self.state))
        # increment step counter
        self.current_step += 1
        # check if episode is terminated
        #terminated = term_fn(action, actual_new_state)
        terminated = self.current_step >= self.episode_length
        # set placeholder for truncated
        truncated = False
        # set placeholder for info
        info = {}
        return torch.FloatTensor([self.state]).flatten(), reward, terminated, truncated, info

    def render(self, mode='human'):
        # simple print rendering
        print(f"Step: {self.current_step}, State: {self.state}")

In [142]:
class SymbolicModel(Model):
    def __init__(self, symbols, population_size, generations, max_length):
        super().__init__("cpu")
        self.reg = SymbolicRegressor(population_size = population_size,
                                     allowed_symbols=symbols,
                                     optimizer_iterations=10,
                                     generations = generations,
                                     n_threads=32,
                                     max_evaluations=int(1e6),
                                     max_length = max_length,
                                     tournament_size = 3)
    def forward(self,x):
        return self.reg.predict(x), None
    
    def loss(self, model_in, target):
        return self.reg.score(model_in, target)
    
    def eval_score(self, model_in, target):
        return self.reg.score(model_in, target)

In [143]:
# Register the custom environment
gym.envs.registration.register(
    id='Simple1DMDP-v0',
    entry_point=Simple1DMDP
)


env = gym.make('Simple1DMDP-v0')

# Reset the environment
state = env.reset()
print(f"Initial State: {state}")

for _ in range(10):
    action = env.action_space.sample()  # Random action
    state, reward, terminated, truncated, info = env.step(action)

    print(f"Action: {action}, State: {state}, Reward: {reward}")
    env.render()

Initial State: (tensor([0.]), {})
Action: [0.21400476], State: tensor([0.2140]), Reward: tensor([0.2408])
Step: 1, State: [0.21400476]
Action: [-0.55617166], State: tensor([-0.3422]), Reward: tensor([-0.6134])
Step: 2, State: [-0.3421669]
Action: [0.30529296], State: tensor([-0.0369]), Reward: tensor([0.9853])
Step: 3, State: [-0.03687394]
Action: [-0.5669092], State: tensor([-0.6038]), Reward: tensor([-0.9720])
Step: 4, State: [-0.60378313]
Action: [-0.28346726], State: tensor([-0.8873]), Reward: tensor([1.0207])
Step: 5, State: [-0.8872504]
Action: [0.23664586], State: tensor([-0.6506]), Reward: tensor([-0.7263])
Step: 6, State: [-0.65060455]
Action: [0.15619661], State: tensor([-0.4944]), Reward: tensor([-1.1784])
Step: 7, State: [-0.49440795]
Action: [0.34170285], State: tensor([-0.1527]), Reward: tensor([0.6039])
Step: 8, State: [-0.1527051]
Action: [0.8935186], State: tensor([0.7408]), Reward: tensor([-0.0738])
Step: 9, State: [0.7408135]
Action: [0.39532644], State: tensor([1.13

In [144]:
seed = 0
env.reset()
rng = np.random.default_rng(seed=seed)
generator = torch.Generator(device=device)
generator.manual_seed(seed)
obs_shape = env.observation_space.shape
act_shape = env.action_space.shape

In [145]:
trial_length = 500
num_trials = 10
ensemble_size = 7

# Everything with "???" indicates an option with a missing value.
# Our utility functions will fill in these details using the 
# environment information
cfg_dict = {
    # dynamics model configuration
    "dynamics_model": {
        "_target_": "mbrl.models.GaussianMLP",
        "device": device,
        "num_layers": 4,
        "ensemble_size": ensemble_size,
        "hid_size": 200,
        "in_size": "???",
        "out_size": "???",
        "deterministic": True,
        "propagation_method": "fixed_model",
        # can also configure activation function for GaussianMLP
        "activation_fn_cfg": {
            "_target_": "torch.nn.SiLU"}
    },
    # options for training the dynamics model
    "algorithm": {
        "learned_rewards": True,
        "target_is_delta": True,
        "normalize": True,
    },
    # these are experiment specific options
    "overrides": {
        "trial_length": trial_length,
        "num_steps": num_trials * trial_length,
        "model_batch_size": 256,
        "validation_ratio": 0.05
    }
}
cfg = omegaconf.OmegaConf.create(cfg_dict)

In [146]:
# Create a 1-D dynamics model for this environment
dynamics_model = common_util.create_one_dim_tr_model(cfg, obs_shape, act_shape)

# Create a gym-like environment to encapsulate the model
model_env = models.ModelEnv(env, dynamics_model, term_fn, reward_fn, generator=generator)

In [147]:
replay_buffer = common_util.create_replay_buffer(cfg, obs_shape, act_shape, rng=rng)

In [148]:
common_util.rollout_agent_trajectories(
    env,
    trial_length, # initial exploration steps
    planning.RandomAgent(env),
    {}, # keyword arguments to pass to agent.act()
    replay_buffer=replay_buffer,
    trial_length=trial_length
)

print("# samples stored", replay_buffer.num_stored)

# samples stored 500


In [149]:
agent_cfg = omegaconf.OmegaConf.create({
    # this class evaluates many trajectories and picks the best one
    "_target_": "mbrl.planning.TrajectoryOptimizerAgent",
    "planning_horizon": 15,
    "replan_freq": 1,
    "verbose": False,
    "action_lb": "???",
    "action_ub": "???",
    # this is the optimizer to generate and choose a trajectory
    "optimizer_cfg": {
        "_target_": "mbrl.planning.CEMOptimizer",
        "num_iterations": 5,
        "elite_ratio": 0.1,
        "population_size": 350,
        "alpha": 0.1,
        "device": device,
        "lower_bound": "???",
        "upper_bound": "???",
        "return_mean_elites": True,
        "clipped_normal": False
    }
})

agent = planning.create_trajectory_optim_agent_for_model(
    model_env,
    agent_cfg,
    num_particles=20
)

In [150]:
train_losses = []
val_scores = []

def train_callback(_model, _total_calls, _epoch, tr_loss, val_score, _best_val):
    train_losses.append(tr_loss)
    val_scores.append(val_score.mean().item())

In [182]:
# Create a trainer for the model
model_trainer = models.ModelTrainer(dynamics_model, optim_lr=7.5e-4, weight_decay=3e-5)

    
# Main PETS loop
all_rewards = [0]
for trial in range(num_trials):
    obs, _ = env.reset()    
    agent.reset()
    
    terminated = False
    total_reward = 0.0
    steps_trial = 0
    while not terminated:
        # --------------- Model Training -----------------
        if steps_trial == 0:
            dynamics_model.update_normalizer(replay_buffer.get_all())  # update normalizer stats
            
            print(replay_buffer.reward.shape)
            
            dataset_train, dataset_val = common_util.get_basic_buffer_iterators(
                replay_buffer,
                batch_size=cfg.overrides.model_batch_size,
                val_ratio=cfg.overrides.validation_ratio,
                ensemble_size=ensemble_size,
                shuffle_each_epoch=True,
                bootstrap_permutes=False,  # build bootstrap dataset using sampling with replacement
            )
                
            model_trainer.train(
                dataset_train, 
                dataset_val=dataset_val, 
                num_epochs=2000, 
                patience=25, 
                callback=train_callback,
                silent=True)
            

        # --- Doing env step using the agent and adding to model dataset ---
        next_obs, reward, terminated, truncated, _ = common_util.step_env_and_add_to_buffer(
            env, obs, agent, {}, replay_buffer)
            
        
        
        obs = next_obs
        total_reward += reward
        steps_trial += 1
        
        if steps_trial == trial_length:
            break
    
    all_rewards.append(total_reward)

(5000,)
(5000,)
(5000,)
(5000,)
(5000,)
(5000,)
(5000,)
(5000,)
(5000,)
(5000,)


In [181]:
replay_buffer.reward

(5000,)

In [170]:
r = torch.from_numpy(np.linspace(-10,10, 91).reshape(-1,1))
with torch.no_grad():
    dynamics_model.model(r)

ValueError: When using propagation='fixed_model', `propagation_indices` must be provided.